<a href="https://colab.research.google.com/github/futurexskill/ml-model-deployment/blob/main/pytorch_save.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [39]:
import torch
import torch.nn as nn
from torch.nn import functional as F

import pandas as pd
import numpy as np

In [40]:
dataset = pd.read_csv('https://raw.githubusercontent.com/futurexskill/ml-model-deployment/main/storepurchasedata.csv')

In [41]:
dataset.sample(3)

,Age,Salary,Purchased
29,47,55000,1
9,27,37000,0
35,56,45000,1


In [42]:
X = dataset.iloc[:, :-1].values
y = dataset.iloc[:,-1].values

In [43]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size =.20,random_state=0)

In [44]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

In [78]:
Xtrain_ = torch.from_numpy(X_train).float()
Xtest_ = torch.from_numpy(X_test).float()


In [110]:
ytrain_ = torch.from_numpy(y_train)
ytest_ = torch.from_numpy(y_test)

In [111]:
Xtrain_.shape, ytrain_.shape

(torch.Size([32, 2]), torch.Size([32]))

In [112]:
Xtest_.shape, ytest_.shape

(torch.Size([8, 2]), torch.Size([8]))

In [113]:
input_size=2
output_size=2
hidden_size=10

In [114]:
class Net(nn.Module):
   def __init__(self):
       super(Net, self).__init__()
       self.fc1 = torch.nn.Linear(input_size, hidden_size)
       self.fc2 = torch.nn.Linear(hidden_size, hidden_size)
       self.fc3 = torch.nn.Linear(hidden_size, output_size)


   def forward(self, X):
       X = torch.sigmoid((self.fc1(X)))
       X = torch.sigmoid((self.fc2(X)))
       X = self.fc3(X)

       return F.log_softmax(X,dim=1)

In [115]:
model = Net()



In [116]:
import torch.optim as optim
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
loss_fn = nn.NLLLoss()

In [117]:
epochs = 100


In [120]:
for epoch in range(epochs):
  optimizer.zero_grad()
  Ypred = model(Xtrain_)
  loss = loss_fn(Ypred,  ytrain_)
  loss.backward()
  optimizer.step()
  print('Epoch',epoch, 'loss',loss.item())

Epoch 0 loss 0.17851537466049194
Epoch 1 loss 0.17786383628845215
Epoch 2 loss 0.17722702026367188
Epoch 3 loss 0.1766042858362198
Epoch 4 loss 0.17599530518054962
Epoch 5 loss 0.17539963126182556
Epoch 6 loss 0.17481674253940582
Epoch 7 loss 0.1742464303970337
Epoch 8 loss 0.17368806898593903
Epoch 9 loss 0.1731412559747696
Epoch 10 loss 0.17260535061359406
Epoch 11 loss 0.17207981646060944
Epoch 12 loss 0.17156410217285156
Epoch 13 loss 0.17105767130851746
Epoch 14 loss 0.1705600917339325
Epoch 15 loss 0.17007090151309967
Epoch 16 loss 0.16958971321582794
Epoch 17 loss 0.16911616921424866
Epoch 18 loss 0.16864977777004242
Epoch 19 loss 0.16819028556346893
Epoch 20 loss 0.16773726046085358
Epoch 21 loss 0.16729041934013367
Epoch 22 loss 0.16684935986995697
Epoch 23 loss 0.16641390323638916
Epoch 24 loss 0.16598378121852875
Epoch 25 loss 0.16555877029895782
Epoch 26 loss 0.16513869166374207
Epoch 27 loss 0.16472342610359192
Epoch 28 loss 0.16431273519992828
Epoch 29 loss 0.163906499743

In [122]:
list(model.parameters())

[Parameter containing:
 tensor([[-0.9864, -1.2837],
         [ 1.4515,  2.3153],
         [-1.1692, -1.9637],
         [ 1.3871,  2.1355],
         [-1.2228, -1.3740],
         [-0.5296, -1.2058],
         [-0.6352, -1.9374],
         [ 0.8996,  1.6957],
         [ 1.2772,  2.2182],
         [ 0.8922,  2.2475]], requires_grad=True), Parameter containing:
 tensor([ 0.6471, -0.4239,  0.4698, -0.4703,  0.6362,  0.5852,  0.4551, -0.4997,
         -0.4275, -0.4124], requires_grad=True), Parameter containing:
 tensor([[-0.6787,  1.0874, -0.9766,  1.0417, -0.6620, -0.5422, -0.9160,  0.5751,
           0.7566,  0.7446],
         [ 0.7517, -0.8780,  0.8212, -0.8919,  0.6313,  0.8368,  0.5486, -0.8961,
          -0.5550, -0.6621],
         [-0.7920,  1.0582, -1.0493,  0.8419, -0.9549, -0.2606, -1.0248,  0.9147,
           0.9827,  0.8524],
         [-1.0450,  1.3990, -1.1639,  1.0068, -0.9116, -1.0928, -1.2452,  1.0440,
           1.2732,  1.2376],
         [-0.0278, -0.3602,  0.5308, -0.2927,  

In [128]:
torch.from_numpy(sc.transform(np.array([[40,20000]]))).float()

tensor([[ 0.0829, -1.4975]])

In [130]:
y_cust_20_40000 = model(torch.from_numpy(sc.transform(np.array([[40,20000]]))).float())
y_cust_20_40000

tensor([[-0.0509, -3.0027]], grad_fn=<LogSoftmaxBackward>)

In [140]:
_, predicted_20_40000 = torch.max(y_cust_20_40000.data,-1)
predicted_20_40000

tensor([0])

In [135]:
y_cust_42_50000 = model(torch.from_numpy(sc.transform(np.array([[42,50000]]))).float())
y_cust_42_50000

tensor([[-1.8194, -0.1769]], grad_fn=<LogSoftmaxBackward>)

In [142]:
_, predicted_42_50000 = torch.max(y_cust_42_50000.data,-1)
predicted_42_50000

tensor([1])

In [143]:
torch.save(model,'customer_buy.pt')

In [144]:
!ls

customer_buy.pt  sample_data


In [145]:
restored_model = torch.load('customer_buy.pt')

In [146]:
y_cust_20_40000 = restored_model(torch.from_numpy(sc.transform(np.array([[40,20000]]))).float())
y_cust_20_40000

tensor([[-0.0509, -3.0027]], grad_fn=<LogSoftmaxBackward>)

In [ ]:
_, predicted_20_40000 = torch.max(y_cust_20_40000.data,-1)
predicted_20_40000

In [137]:
y_cust_42_50000 = restored_model(torch.from_numpy(sc.transform(np.array([[42,50000]]))).float())
y_cust_42_50000

tensor([[-1.8194, -0.1769]], grad_fn=<LogSoftmaxBackward>)

In [147]:
_, predicted_42_50000 = torch.max(y_cust_42_50000.data,-1)
predicted_42_50000

tensor([1])

In [148]:
model.state_dict()

OrderedDict([('fc1.weight', tensor([[-0.9864, -1.2837],
                      [ 1.4515,  2.3153],
                      [-1.1692, -1.9637],
                      [ 1.3871,  2.1355],
                      [-1.2228, -1.3740],
                      [-0.5296, -1.2058],
                      [-0.6352, -1.9374],
                      [ 0.8996,  1.6957],
                      [ 1.2772,  2.2182],
                      [ 0.8922,  2.2475]])),
             ('fc1.bias',
              tensor([ 0.6471, -0.4239,  0.4698, -0.4703,  0.6362,  0.5852,  0.4551, -0.4997,
                      -0.4275, -0.4124])),
             ('fc2.weight',
              tensor([[-0.6787,  1.0874, -0.9766,  1.0417, -0.6620, -0.5422, -0.9160,  0.5751,
                        0.7566,  0.7446],
                      [ 0.7517, -0.8780,  0.8212, -0.8919,  0.6313,  0.8368,  0.5486, -0.8961,
                       -0.5550, -0.6621],
                      [-0.7920,  1.0582, -1.0493,  0.8419, -0.9549, -0.2606, -1.0248,  0.9147,
   

In [150]:
optimizer.state_dict()

{'param_groups': [{'amsgrad': False,
   'betas': (0.9, 0.999),
   'eps': 1e-08,
   'lr': 0.01,
   'params': [0, 1, 2, 3, 4, 5],
   'weight_decay': 0}],
 'state': {0: {'exp_avg': tensor([[ 0.0003,  0.0015],
           [-0.0004, -0.0023],
           [ 0.0005,  0.0021],
           [-0.0004, -0.0021],
           [ 0.0003,  0.0016],
           [ 0.0003,  0.0013],
           [ 0.0005,  0.0018],
           [-0.0004, -0.0018],
           [-0.0005, -0.0021],
           [-0.0006, -0.0021]]),
   'exp_avg_sq': tensor([[1.5701e-06, 1.8931e-06],
           [3.8541e-06, 4.4008e-06],
           [4.5577e-06, 4.5771e-06],
           [3.3392e-06, 3.8481e-06],
           [1.5019e-06, 1.9912e-06],
           [1.2225e-06, 1.3598e-06],
           [1.6402e-06, 1.8860e-06],
           [2.1237e-06, 2.4479e-06],
           [3.5023e-06, 3.8504e-06],
           [3.2240e-06, 3.2902e-06]]),
   'step': 200},
  1: {'exp_avg': tensor([ 2.9473e-05, -9.1397e-05,  7.7799e-05, -8.9245e-05,  4.7826e-05,
           -6.7421e-

In [165]:
torch.save(model.state_dict(),'customer_buy_state_dict')

In [166]:
!ls

customer_buy.pt		 customer_buy_state_dict.pt
customer_buy_state_dict  sample_data


In [167]:
new_predictor = Net()

In [168]:
new_predictor.load_state_dict(torch.load('customer_buy_state_dict'))

<All keys matched successfully>

In [170]:
y_cust_20_40000 = new_predictor(torch.from_numpy(sc.transform(np.array([[40,20000]]))).float())
y_cust_20_40000

tensor([[-0.0509, -3.0027]], grad_fn=<LogSoftmaxBackward>)

In [169]:
y_cust_42_50000 = new_predictor(torch.from_numpy(sc.transform(np.array([[42,50000]]))).float())
y_cust_42_50000

tensor([[-1.8194, -0.1769]], grad_fn=<LogSoftmaxBackward>)